In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# *Installing Required Libraries*

In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manyli

In [ ]:
%cd '/content/drive/MyDrive/'

/content/drive/MyDrive


In [ ]:
!unzip /content/data.zip

Archive:  /content/data.zip
  inflating: data/val.csv            
  inflating: data/train.csv          
  inflating: data/test.csv           
  inflating: data/final_val.csv      
  inflating: data/final_test.csv     
  inflating: data/final_train.csv    


# *Importing libraries*

In [ ]:
import json
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
import torch
from datasets import Dataset
from datasets import load_dataset
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForMaskedLM,AutoModelForSequenceClassification
from transformers import TrainerCallback, EarlyStoppingCallback
from transformers import TrainingArguments, Trainer

In [ ]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
val_df = pd.read_csv("val.csv")


train_df = train_df.drop(columns=['Num_Tokens','Num_Sentences'])
test_df = test_df.drop(columns=['Num_Tokens','Num_Sentences'])
val_df = val_df.drop(columns=['Num_Tokens','Num_Sentences'])

In [ ]:
train_df['labels'].value_counts()

,count
labels,
0,2319
1,741


In [ ]:
val_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [ ]:
test_df['labels'].value_counts()

,count
labels,
0,774
1,247


In [ ]:
train_df.to_csv('final_train.csv',index=False, encoding='utf-8')
test_df.to_csv('final_test.csv',index=False, encoding='utf-8')
val_df.to_csv('final_val.csv',index=False, encoding='utf-8')

## *Dataset loading as HuggingFace Trainer*

In [ ]:
dataset = load_dataset('csv', data_files={'train': "final_train.csv",
                                              'val':"final_val.csv",'test':"final_test.csv"})

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['clean_text', 'labels', 'corrected_text', 'language_tags'],
        num_rows: 3060
    })
    val: Dataset({
        features: ['clean_text', 'labels', 'corrected_text', 'language_tags'],
        num_rows: 1021
    })
    test: Dataset({
        features: ['clean_text', 'labels', 'corrected_text', 'language_tags'],
        num_rows: 1021
    })
})

## Interleaved Word-Language Approach

In [ ]:
def interleaved_preprocess(sentences):
    texts = sentences['corrected_text']
    tags = sentences['language_tags']

    processed_texts = []

    for text, tag in zip(texts, tags):
        words = text.split()
        tags_list = tag.split(", ")
        interleaved_text = " ".join([f"{word} [{tag}]" for word, tag in zip(words, tags_list)])
        processed_texts.append(interleaved_text)

    return {"interleaved_text_tags": processed_texts}

dataset = dataset.map(interleaved_preprocess, batched=True)

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [ ]:
print(dataset['train'][0]['interleaved_text_tags'])

hindu [EN] to [HI] kuch [HI] hai [HI] par [HI] tum [HI] to [HI] pori [HI] duniya [HI] mai [HI] gandgi [HI] macha [HI] rhe [HI] ho [HI] or [EN] bat [EN] rhi [HI] bhagwe [HI] ki [HI] to [HI] tumhe [HI] aba [EN] ke [HI] abe [HI] bhagwa [HI] hi [HI] the [HI] talvar [HI] ke [HI] dar [HI] se [HI] salvaar [HI] khol [HI] di [HI] unhone [HI] isme [HI] tumhari [HI] koi [HI] galti [HI] nhi [HI] hai [HI]


In [ ]:
dataset['train'][0]

{'clean_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvar khol di unhone isme tumhari koi galti nhi hai',
 'labels': 1,
 'corrected_text': 'hindu to kuch hai par tum to pori duniya mai gandgi macha rhe ho or bat rhi bhagwe ki to tumhe aba ke abe bhagwa hi the talvar ke dar se salvaar khol di unhone isme tumhari koi galti nhi hai',
 'language_tags': 'EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, EN, EN, HI, HI, HI, HI, HI, EN, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI',
 'interleaved_text_tags': 'hindu [EN] to [HI] kuch [HI] hai [HI] par [HI] tum [HI] to [HI] pori [HI] duniya [HI] mai [HI] gandgi [HI] macha [HI] rhe [HI] ho [HI] or [EN] bat [EN] rhi [HI] bhagwe [HI] ki [HI] to [HI] tumhe [HI] aba [EN] ke [HI] abe [HI] bhagwa [HI] hi [HI] the [HI] talvar [HI] ke [HI] dar [HI] se [HI] salvaar [HI] khol [HI] di [HI] unhone [HI] isme [HI] tumhari [

In [ ]:
dataset['val'][0]

{'clean_text': 'hindu rastra ke ruzhan miya bhai ko kast hai chalo hindu is bat pe mast hai',
 'labels': 1,
 'corrected_text': 'hindu rastra ke rujhan miya bhai ko kast hai chalo hindu is bat pe mast hai',
 'language_tags': 'EN, EN, HI, HI, HI, HI, HI, HI, HI, HI, EN, HI, EN, HI, EN, HI',
 'interleaved_text_tags': 'hindu [EN] rastra [EN] ke [HI] rujhan [HI] miya [HI] bhai [HI] ko [HI] kast [HI] hai [HI] chalo [HI] hindu [EN] is [HI] bat [EN] pe [HI] mast [EN] hai [HI]'}

In [ ]:
dataset['test'][0]

{'clean_text': 'jai hind , in namuno ko bat bhi karni nahi ati hai , inko pension chahiye',
 'labels': 0,
 'corrected_text': 'jai hind , in namuno ko bat bhi karni nahi ati hai , inko pension chahiye',
 'language_tags': 'HI, EN, OOV, EN, HI, HI, EN, HI, HI, HI, HI, HI, OOV, HI, EN, HI',
 'interleaved_text_tags': 'jai [HI] hind [EN] , [OOV] in [EN] namuno [HI] ko [HI] bat [EN] bhi [HI] karni [HI] nahi [HI] ati [HI] hai [HI] , [OOV] inko [HI] pension [EN] chahiye [HI]'}

# For 5000 rows dataset with clean_text

# *Tokenization as per Hing-MBERT*

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-mbert")
print(len(tokenizer))
special_tokens = ["[EN]", "[HI]", "[OOV]"]
tokenizer.add_tokens(special_tokens)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/327 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.92M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

119547


3

In [ ]:
len(tokenizer)

119550

In [ ]:
mbert_dataset = dataset.map(
    lambda example: tokenizer(example['interleaved_text_tags'],max_length=194,padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)
mbert_dataset = mbert_dataset.remove_columns(["clean_text","corrected_text","language_tags","interleaved_text_tags"])
mbert_dataset.set_format("torch")

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [ ]:
mbert_dataset['train'][0]

{'labels': tensor(1),
 'input_ids': tensor([   101,  19911,  11460, 119547,  10114, 119548,  15694,  10269, 119548,
          13080, 119548,  10248, 119548,  13055,  10147, 119548,  10114, 119548,
          10183,  10116, 119548,  23145,  14990, 119548,  10737, 119548,  13412,
          10162,  11210, 119548, 109664,  10113, 119548,    186,  11643, 119548,
          13173, 119548,  10345, 119547,  11519, 119547,    186,  11924, 119548,
            170,  10921,  10240,  12577, 119548,  10879, 119548,  10114, 119548,
          13055,  34927,  10112, 119548,  11357,  10113, 119547,  11163, 119548,
          11357,  10112, 119548,    170,  10921,  10240,  11037, 119548,  11520,
         119548,  10105, 119548,  13675,  16648, 119548,  11163, 119548,  12923,
         119548,  10126, 119548,  86521,  10354, 119548,    179,  32631, 119548,
          10120, 119548,  10119,  50198, 119548,  10124,  10627, 119548,  13055,
          34927,  12476, 119548,  11252,  10116, 119548,  37188,  10116, 1

In [ ]:
print("Batch Decode:")
print(tokenizer.batch_decode(mbert_dataset['train'][0]['input_ids']))

Batch Decode:
['[CLS]', 'hin', '##du', '[EN]', 'to', '[HI]', 'ku', '##ch', '[HI]', 'hai', '[HI]', 'par', '[HI]', 'tu', '##m', '[HI]', 'to', '[HI]', 'por', '##i', '[HI]', 'dun', '##iya', '[HI]', 'mai', '[HI]', 'gan', '##d', '##gi', '[HI]', 'mach', '##a', '[HI]', 'r', '##he', '[HI]', 'ho', '[HI]', 'or', '[EN]', 'bat', '[EN]', 'r', '##hi', '[HI]', 'b', '##ha', '##g', '##we', '[HI]', 'ki', '[HI]', 'to', '[HI]', 'tu', '##mh', '##e', '[HI]', 'ab', '##a', '[EN]', 'ke', '[HI]', 'ab', '##e', '[HI]', 'b', '##ha', '##g', '##wa', '[HI]', 'hi', '[HI]', 'the', '[HI]', 'tal', '##var', '[HI]', 'ke', '[HI]', 'dar', '[HI]', 'se', '[HI]', 'salva', '##ar', '[HI]', 'k', '##hol', '[HI]', 'di', '[HI]', 'un', '##hone', '[HI]', 'is', '##me', '[HI]', 'tu', '##mh', '##ari', '[HI]', 'ko', '##i', '[HI]', 'galt', '##i', '[HI]', 'n', '##hi', '[HI]', 'hai', '[HI]', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

In [ ]:
cls = [tokenizer.cls_token_id]
sep = [tokenizer.sep_token_id]

input_str = dataset['train'][0]['interleaved_text_tags']
# Tokenization happens in a few steps:
input_tokens = tokenizer.tokenize(input_str)
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)
input_ids_special_tokens = cls + input_ids + sep

decoded_str = tokenizer.decode(input_ids_special_tokens)

print("start:                ", input_str)
print("tokenize:             ", input_tokens)
print("convert_tokens_to_ids:", input_ids)
print("add special tokens:   ", input_ids_special_tokens)
print("--------")
print("decode:               ", decoded_str)

start:                 hindu [EN] to [HI] kuch [HI] hai [HI] par [HI] tum [HI] to [HI] pori [HI] duniya [HI] mai [HI] gandgi [HI] macha [HI] rhe [HI] ho [HI] or [EN] bat [EN] rhi [HI] bhagwe [HI] ki [HI] to [HI] tumhe [HI] aba [EN] ke [HI] abe [HI] bhagwa [HI] hi [HI] the [HI] talvar [HI] ke [HI] dar [HI] se [HI] salvaar [HI] khol [HI] di [HI] unhone [HI] isme [HI] tumhari [HI] koi [HI] galti [HI] nhi [HI] hai [HI]
tokenize:              ['hin', '##du', '[EN]', 'to', '[HI]', 'ku', '##ch', '[HI]', 'hai', '[HI]', 'par', '[HI]', 'tu', '##m', '[HI]', 'to', '[HI]', 'por', '##i', '[HI]', 'dun', '##iya', '[HI]', 'mai', '[HI]', 'gan', '##d', '##gi', '[HI]', 'mach', '##a', '[HI]', 'r', '##he', '[HI]', 'ho', '[HI]', 'or', '[EN]', 'bat', '[EN]', 'r', '##hi', '[HI]', 'b', '##ha', '##g', '##we', '[HI]', 'ki', '[HI]', 'to', '[HI]', 'tu', '##mh', '##e', '[HI]', 'ab', '##a', '[EN]', 'ke', '[HI]', 'ab', '##e', '[HI]', 'b', '##ha', '##g', '##wa', '[HI]', 'hi', '[HI]', 'the', '[HI]', 'tal', '##var', '[HI

# *Calculating Weights for Labels with Imbalanced Dataset*

In [ ]:
# Define all possible class labels
class_labels = np.unique(mbert_dataset['train']['labels'])

# Calculate class weights
labels = mbert_dataset['train']['labels']
class_weights = compute_class_weight(class_weight='balanced', classes=class_labels, y=labels.numpy())
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(class_weights)

tensor([0.6598, 2.0648])


In [ ]:
class_labels

array([0, 1])

# **Weighted Loss Trainer for Imbalanced Datset**

In [ ]:
from transformers import TrainingArguments, Trainer
# Define a custom Trainer class to include class weights
class WeightedLossTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels").to(model.device)  # Ensure labels are on the same device as model
        inputs = {k: v.to(model.device) for k, v in inputs.items()}  # Ensure all inputs are on the same device as model
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # Compute weighted loss
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(model.device))  # Move class_weights to the same device as model
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# *Function: Compute metrics for all labels*

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # Additional metrics
    recall_micro = recall_score(labels, predictions, average='micro')
    f1_micro = f1_score(labels, predictions, average='micro')

    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')

    recall_positive = recall_score(labels, predictions, pos_label=1)
    f1_positive = f1_score(labels, predictions, pos_label=1)

    recall_negative = recall_score(labels, predictions, pos_label=0)
    f1_negative = f1_score(labels, predictions, pos_label=0)

    return {
        'accuracy': accuracy,
        'precision_weighted': precision,
        'recall_weighted': recall,
        'f1_weighted': f1,
        'recall_micro': recall_micro,
        'f1_micro': f1_micro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'recall_positive': recall_positive,
        'f1_positive': f1_positive,
        'recall_negative': recall_negative,
        'f1_negative': f1_negative
    }

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-mbert", num_labels=2)

model.resize_token_embeddings(len(tokenizer))

arguments = TrainingArguments(
    output_dir="sample_HingMBert_trainer_5k_itags",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision = precision_score(labels, predictions, average='weighted')
    recall = recall_score(labels, predictions, average='weighted')
    f1 = f1_score(labels, predictions, average='weighted')
    accuracy = accuracy_score(labels, predictions)

    # Additional metrics
    recall_micro = recall_score(labels, predictions, average='micro')
    f1_micro = f1_score(labels, predictions, average='micro')

    recall_macro = recall_score(labels, predictions, average='macro')
    f1_macro = f1_score(labels, predictions, average='macro')

    recall_positive = recall_score(labels, predictions, pos_label=1)
    f1_positive = f1_score(labels, predictions, pos_label=1)

    recall_negative = recall_score(labels, predictions, pos_label=0)
    f1_negative = f1_score(labels, predictions, pos_label=0)

    return {
        'accuracy': accuracy,
        'precision_weighted': precision,
        'recall_weighted': recall,
        'f1_weighted': f1,
        'recall_micro': recall_micro,
        'f1_micro': f1_micro,
        'recall_macro': recall_macro,
        'f1_macro': f1_macro,
        'recall_positive': recall_positive,
        'f1_positive': f1_positive,
        'recall_negative': recall_negative,
        'f1_negative': f1_negative
    }


trainer = WeightedLossTrainer(
    model=model,
    args=arguments,
    train_dataset=mbert_dataset['train'],
    eval_dataset=mbert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-mbert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# *Logger Function for early Stopping and logging*

In [ ]:
class LoggingCallback(TrainerCallback):
    def __init__(self, log_path):
        self.log_path = log_path
    # will call on_log on each logging step, specified by TrainerArguement. (i.e TrainerArguement.logginng_step)
    def on_log(self, args, state, control, logs=None, **kwargs):
        _ = logs.pop("total_flos", None)
        if state.is_local_process_zero:
            with open(self.log_path, "a") as f:
                f.write(json.dumps(logs) + "\n")


In [ ]:
trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.01))
trainer.add_callback(LoggingCallback("sample_HingMBert_trainer_5k_itags/log.jsonl"))

In [ ]:
# train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.678004,0.743389,0.725645,0.743389,0.732464,0.743389,0.743389,0.614357,0.621745,0.364372,0.407240,0.864341,0.836250
2,No log,0.543770,0.707150,0.799017,0.707150,0.727597,0.707150,0.707150,0.737933,0.673448,0.797571,0.568543,0.678295,0.778354
3,No log,0.548783,0.781587,0.782496,0.781587,0.782034,0.781587,0.781587,0.704331,0.703486,0.554656,0.551308,0.854005,0.855663
4,No log,0.578060,0.798237,0.800512,0.798237,0.799321,0.798237,0.798237,0.730474,0.727938,0.599190,0.589641,0.861757,0.866234


TrainOutput(global_step=192, training_loss=0.574931820233663, metrics={'train_runtime': 571.9556, 'train_samples_per_second': 53.501, 'train_steps_per_second': 0.839, 'train_loss': 0.574931820233663, 'epoch': 4.0})

In [ ]:
mbert_results = trainer.evaluate()

In [ ]:
print(mbert_results)

{'eval_loss': 0.5437696576118469, 'eval_accuracy': 0.7071498530852106, 'eval_precision_weighted': 0.7990171880851121, 'eval_recall_weighted': 0.7071498530852106, 'eval_f1_weighted': 0.7275967393891289, 'eval_recall_micro': 0.7071498530852106, 'eval_f1_micro': 0.7071498530852106, 'eval_recall_macro': 0.73793271192292, 'eval_f1_macro': 0.6734484525440789, 'eval_recall_positive': 0.7975708502024291, 'eval_f1_positive': 0.5685425685425686, 'eval_recall_negative': 0.6782945736434108, 'eval_f1_negative': 0.7783543365455894, 'eval_runtime': 10.8585, 'eval_samples_per_second': 94.028, 'eval_steps_per_second': 5.894, 'epoch': 4.0}


# **Hing-MBERT Test results**

In [ ]:
test_results_mbert = trainer.predict(mbert_dataset['test'])

In [ ]:
test_results_mbert

PredictionOutput(predictions=array([[ 0.60677236, -0.52389973],
       [ 0.20549195, -0.26495123],
       [ 0.6188637 , -0.53422517],
       ...,
       [-0.53147197,  0.7012571 ],
       [-0.75330764,  0.87940377],
       [ 0.2748433 , -0.33172897]], dtype=float32), label_ids=array([0, 0, 0, ..., 1, 0, 0]), metrics={'test_loss': 0.5651074647903442, 'test_accuracy': 0.6797257590597453, 'test_precision_weighted': 0.7798147160044336, 'test_recall_weighted': 0.6797257590597453, 'test_f1_weighted': 0.7023811220126168, 'test_recall_micro': 0.6797257590597453, 'test_f1_micro': 0.6797257590597453, 'test_recall_macro': 0.7088184832982874, 'test_f1_macro': 0.6457964003246391, 'test_recall_positive': 0.7651821862348178, 'test_f1_positive': 0.5361702127659574, 'test_recall_negative': 0.6524547803617571, 'test_f1_negative': 0.7554225878833208, 'test_runtime': 10.8195, 'test_samples_per_second': 94.366, 'test_steps_per_second': 5.915})

In [ ]:
test_results_mbert.predictions.argmax(axis=1)

array([0, 0, 0, ..., 1, 1, 0])

In [ ]:
test_df['itags_predicted_labels_5k_hingMbert'] = test_results_mbert.predictions.argmax(axis=1)

In [ ]:
print(test_results_mbert.metrics)

{'test_loss': 0.5651074647903442, 'test_accuracy': 0.6797257590597453, 'test_precision_weighted': 0.7798147160044336, 'test_recall_weighted': 0.6797257590597453, 'test_f1_weighted': 0.7023811220126168, 'test_recall_micro': 0.6797257590597453, 'test_f1_micro': 0.6797257590597453, 'test_recall_macro': 0.7088184832982874, 'test_f1_macro': 0.6457964003246391, 'test_recall_positive': 0.7651821862348178, 'test_f1_positive': 0.5361702127659574, 'test_recall_negative': 0.6524547803617571, 'test_f1_negative': 0.7554225878833208, 'test_runtime': 10.8195, 'test_samples_per_second': 94.366, 'test_steps_per_second': 5.915}


# **Hing BERT Training**

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
## dataset pre processing and alignment
# dataset = load_dataset('csv', data_files={'train': "data/final_train.csv",
#                                               'val':"data/final_val.csv",'test':"data/final_test.csv"})

bert_tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/hing-bert")
bert_tokenizer.add_tokens(special_tokens)
bert_dataset = dataset.map(
    lambda example: bert_tokenizer(example['interleaved_text_tags'], max_length=194, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)


bert_dataset = bert_dataset.remove_columns(["clean_text","corrected_text","language_tags","interleaved_text_tags"])
bert_dataset.set_format("torch")

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

In [ ]:
## model training
bert_model = AutoModelForSequenceClassification.from_pretrained("l3cube-pune/hing-bert")

bert_model.resize_token_embeddings(len(bert_tokenizer))
arguments = TrainingArguments(
    output_dir="sample_Hing_Bert_trainer_5k_itags",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    optim = 'adamw_torch',
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=5e-5,
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

bert_trainer = WeightedLossTrainer(
    model=bert_model,
    args=arguments,
    train_dataset=bert_dataset['train'],
    eval_dataset=bert_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=bert_tokenizer,
    compute_metrics=compute_metrics
)

bert_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=2, early_stopping_threshold=0.01))
bert_trainer.add_callback(LoggingCallback("sample_Hing_Bert_trainer_5k_itags/log.jsonl"))

config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/hing-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# train the model
bert_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.689378,0.757101,0.574506,0.757101,0.653284,0.757101,0.757101,0.499354,0.430881,0.000000,0.000000,0.998708,0.861761
2,No log,0.651066,0.742409,0.731648,0.742409,0.736315,0.742409,0.742409,0.627494,0.632702,0.404858,0.431965,0.850129,0.833439
3,No log,0.570603,0.741430,0.775655,0.741430,0.753186,0.741430,0.741430,0.708167,0.682785,0.643725,0.546392,0.772610,0.819178
4,No log,0.585186,0.767875,0.780439,0.767875,0.773120,0.767875,0.767875,0.709070,0.698411,0.595142,0.553672,0.822997,0.843150
5,No log,0.557832,0.670911,0.787541,0.670911,0.694516,0.670911,0.670911,0.715409,0.642248,0.801619,0.540984,0.629199,0.743511
6,No log,0.657641,0.729677,0.769066,0.729677,0.742916,0.729677,0.729677,0.699037,0.671719,0.639676,0.533784,0.758398,0.809655
7,No log,0.782977,0.724780,0.769450,0.724780,0.739231,0.724780,0.724780,0.699942,0.669361,0.651822,0.533997,0.748062,0.804726


TrainOutput(global_step=336, training_loss=0.5054216839018322, metrics={'train_runtime': 894.1259, 'train_samples_per_second': 34.223, 'train_steps_per_second': 0.537, 'train_loss': 0.5054216839018322, 'epoch': 7.0})

In [ ]:
bert_eval_results = bert_trainer.evaluate()
print(bert_eval_results)

{'eval_loss': 0.5578324198722839, 'eval_accuracy': 0.6709108716944172, 'eval_precision_weighted': 0.787541283756606, 'eval_recall_weighted': 0.6709108716944172, 'eval_f1_weighted': 0.6945159778796199, 'eval_recall_micro': 0.6709108716944172, 'eval_f1_micro': 0.6709108716944172, 'eval_recall_macro': 0.7154091998033247, 'eval_f1_macro': 0.6422475284695282, 'eval_recall_positive': 0.8016194331983806, 'eval_f1_positive': 0.540983606557377, 'eval_recall_negative': 0.6291989664082688, 'eval_f1_negative': 0.7435114503816793, 'eval_runtime': 10.9412, 'eval_samples_per_second': 93.317, 'eval_steps_per_second': 5.849, 'epoch': 7.0}


# **Test results for Hing-BERT**

In [ ]:
bert_results = bert_trainer.predict(bert_dataset['test'])
test_df['itags_predicted_labels_5k_hingBert'] = bert_results.predictions.argmax(axis=1)
print(bert_results.metrics)

{'test_loss': 0.550614595413208, 'test_accuracy': 0.682664054848188, 'test_precision_weighted': 0.8015698274185129, 'test_recall_weighted': 0.682664054848188, 'test_f1_weighted': 0.7054338174583823, 'test_recall_micro': 0.682664054848188, 'test_f1_micro': 0.682664054848188, 'test_recall_macro': 0.7341875111152957, 'test_f1_macro': 0.655848425327918, 'test_recall_positive': 0.8340080971659919, 'test_f1_positive': 0.5597826086956522, 'test_recall_negative': 0.6343669250645995, 'test_f1_negative': 0.7519142419601837, 'test_runtime': 10.8909, 'test_samples_per_second': 93.748, 'test_steps_per_second': 5.876}


# **Muril cased training**

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
muril_tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
print(len(muril_tokenizer))

## Adding special tokens
special_tokens = ["[EN]", "[HI]", "[OOV]"]
muril_tokenizer.add_tokens(special_tokens)
print(len(muril_tokenizer))
muril_dataset = dataset.map(
    lambda example: muril_tokenizer(example['interleaved_text_tags'], max_length=194, padding='max_length', truncation=True),
    batched=True,
    batch_size=64
)

muril_dataset = muril_dataset.remove_columns(["clean_text","corrected_text","language_tags","interleaved_text_tags"])
muril_dataset.set_format("torch")

muril_model = AutoModelForSequenceClassification.from_pretrained("google/muril-base-cased",num_labels=2)

muril_model.resize_token_embeddings(len(muril_tokenizer))

arguments = TrainingArguments(
    output_dir="sample_Hing_muril_trainer_5k_itags",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=1e-4,
    warmup_steps=200,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

muril_trainer = WeightedLossTrainer(
    model=muril_model,
    args=arguments,
    train_dataset=muril_dataset['train'],
    eval_dataset=muril_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=muril_tokenizer,
    compute_metrics=compute_metrics
)

muril_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
muril_trainer.add_callback(LoggingCallback("sample_Hing_muril_trainer_5k_itags/log.jsonl"))

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

197285
197288


Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/muril-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
muril_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.693260,0.241920,0.058525,0.241920,0.094249,0.241920,0.241920,0.500000,0.194795,1.000000,0.389590,0.000000,0.000000
2,No log,0.693025,0.758080,0.574686,0.758080,0.653765,0.758080,0.758080,0.500000,0.431198,0.000000,0.000000,1.000000,0.862396
3,No log,0.693030,0.758080,0.574686,0.758080,0.653765,0.758080,0.758080,0.500000,0.431198,0.000000,0.000000,1.000000,0.862396
4,No log,0.692970,0.758080,0.574686,0.758080,0.653765,0.758080,0.758080,0.500000,0.431198,0.000000,0.000000,1.000000,0.862396
5,No log,0.692670,0.758080,0.574686,0.758080,0.653765,0.758080,0.758080,0.500000,0.431198,0.000000,0.000000,1.000000,0.862396


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning

TrainOutput(global_step=240, training_loss=0.6932713190714518, metrics={'train_runtime': 894.0221, 'train_samples_per_second': 34.227, 'train_steps_per_second': 0.537, 'train_loss': 0.6932713190714518, 'epoch': 5.0})

In [ ]:
muril_eval_results = muril_trainer.evaluate()
print(muril_eval_results)

{'eval_loss': 0.6926696300506592, 'eval_accuracy': 0.7580803134182175, 'eval_precision_weighted': 0.5746857615922628, 'eval_recall_weighted': 0.7580803134182175, 'eval_f1_weighted': 0.6537650836609474, 'eval_recall_micro': 0.7580803134182175, 'eval_f1_micro': 0.7580803134182175, 'eval_recall_macro': 0.5, 'eval_f1_macro': 0.43119777158774375, 'eval_recall_positive': 0.0, 'eval_f1_positive': 0.0, 'eval_recall_negative': 1.0, 'eval_f1_negative': 0.8623955431754875, 'eval_runtime': 11.5356, 'eval_samples_per_second': 88.508, 'eval_steps_per_second': 5.548, 'epoch': 5.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
muril_results = muril_trainer.predict(muril_dataset['test'])
test_df['itags_predicted_labels_5k_muril'] = muril_results.predictions.argmax(axis=1)
print(muril_results.metrics)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 0.6925448179244995, 'test_accuracy': 0.7580803134182175, 'test_precision_weighted': 0.5746857615922628, 'test_recall_weighted': 0.7580803134182175, 'test_f1_weighted': 0.6537650836609474, 'test_recall_micro': 0.7580803134182175, 'test_f1_micro': 0.7580803134182175, 'test_recall_macro': 0.5, 'test_f1_macro': 0.43119777158774375, 'test_recall_positive': 0.0, 'test_f1_positive': 0.0, 'test_recall_negative': 1.0, 'test_f1_negative': 0.8623955431754875, 'test_runtime': 12.1211, 'test_samples_per_second': 84.233, 'test_steps_per_second': 5.28}


## XLMR training

In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
xlmr_tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
xlmr_tokenizer.add_tokens(special_tokens)

xlmr_dataset = dataset.map(
    lambda example: xlmr_tokenizer(example['interleaved_text_tags'], max_length=194, padding='max_length', truncation=True),
    batched=True,
    batch_size=16
)

xlmr_dataset = xlmr_dataset.remove_columns(["clean_text","corrected_text","language_tags","interleaved_text_tags"])
xlmr_dataset.set_format("torch")

xlmr_model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-base",num_labels=2)

xlmr_model.resize_token_embeddings(len(xlmr_tokenizer))

arguments = TrainingArguments(
    output_dir="sample_Hing_xlmr_trainer_5k_itags",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy= "epoch",
    learning_rate=2e-4,
    warmup_steps=200,                # number of warmup steps for learning rate scheduler
    weight_decay=0.001,               # strength of weight decay
    load_best_model_at_end=True,
     metric_for_best_model='eval_loss',  # Define the metric for early stopping
    greater_is_better=False,  # Set to False because we want to minimize the loss
    save_total_limit=1,
    seed=224
)

xlmr_trainer = WeightedLossTrainer(
    model=xlmr_model,
    args=arguments,
    train_dataset=xlmr_dataset['train'],
    eval_dataset=xlmr_dataset['val'], # change to test when you do your final evaluation!
    tokenizer=xlmr_tokenizer,
    compute_metrics=compute_metrics
)

xlmr_trainer.add_callback(EarlyStoppingCallback(early_stopping_patience=4, early_stopping_threshold=0.01))
xlmr_trainer.add_callback(LoggingCallback("sample_Hing_xlmr_trainer_5k_itags/log.jsonl"))

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/3060 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
xlmr_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision Weighted,Recall Weighted,F1 Weighted,Recall Micro,F1 Micro,Recall Macro,F1 Macro,Recall Positive,F1 Positive,Recall Negative,F1 Negative
1,No log,0.692377,0.758080,0.574686,0.758080,0.653765,0.758080,0.758080,0.500000,0.431198,0.000000,0.000000,1.000000,0.862396
2,No log,0.692937,0.758080,0.574686,0.758080,0.653765,0.758080,0.758080,0.500000,0.431198,0.000000,0.000000,1.000000,0.862396
3,No log,0.714148,0.241920,0.058525,0.241920,0.094249,0.241920,0.241920,0.500000,0.194795,1.000000,0.389590,0.000000,0.000000
4,No log,0.700995,0.241920,0.058525,0.241920,0.094249,0.241920,0.241920,0.500000,0.194795,1.000000,0.389590,0.000000,0.000000
5,No log,0.692752,0.758080,0.574686,0.758080,0.653765,0.758080,0.758080,0.500000,0.431198,0.000000,0.000000,1.000000,0.862396


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning

TrainOutput(global_step=240, training_loss=0.7018045425415039, metrics={'train_runtime': 799.7579, 'train_samples_per_second': 38.262, 'train_steps_per_second': 0.6, 'train_loss': 0.7018045425415039, 'epoch': 5.0})

In [ ]:
test_df = pd.read_csv("/content/final_test_5k_itags_v2.csv")

In [ ]:
xlmr_eval_results = xlmr_trainer.evaluate()
print(xlmr_eval_results)

{'eval_loss': 0.6923773288726807, 'eval_accuracy': 0.7580803134182175, 'eval_precision_weighted': 0.5746857615922628, 'eval_recall_weighted': 0.7580803134182175, 'eval_f1_weighted': 0.6537650836609474, 'eval_recall_micro': 0.7580803134182175, 'eval_f1_micro': 0.7580803134182175, 'eval_recall_macro': 0.5, 'eval_f1_macro': 0.43119777158774375, 'eval_recall_positive': 0.0, 'eval_f1_positive': 0.0, 'eval_recall_negative': 1.0, 'eval_f1_negative': 0.8623955431754875, 'eval_runtime': 11.898, 'eval_samples_per_second': 85.813, 'eval_steps_per_second': 5.379, 'epoch': 5.0}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
xlmr_results = xlmr_trainer.predict(xlmr_dataset['test'])
test_df['itags_predicted_labels_5k_xlmr'] = xlmr_results.predictions.argmax(axis=1)
print(xlmr_results.metrics)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'test_loss': 0.6921034455299377, 'test_accuracy': 0.7580803134182175, 'test_precision_weighted': 0.5746857615922628, 'test_recall_weighted': 0.7580803134182175, 'test_f1_weighted': 0.6537650836609474, 'test_recall_micro': 0.7580803134182175, 'test_f1_micro': 0.7580803134182175, 'test_recall_macro': 0.5, 'test_f1_macro': 0.43119777158774375, 'test_recall_positive': 0.0, 'test_f1_positive': 0.0, 'test_recall_negative': 1.0, 'test_f1_negative': 0.8623955431754875, 'test_runtime': 12.1602, 'test_samples_per_second': 83.963, 'test_steps_per_second': 5.263}


In [ ]:
test_df

,clean_text,labels,corrected_text,language_tags,itags_predicted_labels_5k_hingMbert,itags_predicted_labels_5k_hingBert,itags_predicted_labels_5k_xlmr,itags_predicted_labels_5k_muril
0,"jai hind , in namuno ko bat bhi karni nahi ati...",0,"jai hind , in namuno ko bat bhi karni nahi ati...","HI, EN, OOV, EN, HI, HI, EN, HI, HI, HI, HI, H...",0,0,0,0
1,sasta nasha karna band karo bhaikis mandir me ...,0,sasta nasha karna band karo bhakts mandir me e...,"HI, HI, HI, HI, HI, EN, HI, HI, EN, HI, HI, EN...",0,1,0,0
2,ap insaniyat wali patrakarita karte hai ap bah...,0,ap insaniayat wali pratrakarita karte hai ap b...,"HI, HI, HI, HI, HI, HI, HI, HI, EN, HI, HI, HI...",0,0,0,0
3,didi kya likh rhi ho kya smjha rhi ho pta hi n...,0,didi kya likh rhi ho kya smjha rhi ho pta hi n...,"HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI, HI...",0,0,0,0
4,humesa christian ka hen against kerney mein bu...,1,humesa christian ka hen against kerney mein bu...,"HI, EN, HI, HI, EN, EN, HI, EN, HI, HI, EN, HI...",1,0,0,0
...,...,...,...,...,...,...,...,...
1016,"yeh khel to north east me regular chalta hai ,...",1,"yeh khel to north east me regular chalta hai ,...","HI, HI, HI, EN, EN, HI, EN, HI, HI, OOV, HI, H...",1,1,0,0
1017,"bjp kerela se har rhi hai to , ab ye faltu ki ...",0,"bjp kerela se har rhi hai to , ab ye faltu ki ...","EN, EN, HI, HI, HI, HI, HI, OOV, HI, HI, HI, H...",0,0,0,0
1018,"amir khan , naserudin shah , shahrukh khan jai...",1,"amir khan , nasedi shah , shahrukh khan jaise ...","HI, EN, OOV, HI, HI, OOV, HI, EN, HI, HI, HI, ...",1,1,0,0
1019,mai ek muslim hu hatwao aese mazaro ko ye isla...,0,mai ek muslim hu hatwao aese mazaro ko ye isla...,"HI, HI, EN, HI, HI, HI, HI, HI, HI, EN, HI, HI...",1,1,0,0


In [ ]:
test_df.to_csv('final_test_5k_itags_v2.csv',index=False, encoding='utf-8')